<a href="https://colab.research.google.com/github/bshakya77/GRI-Research-Repo/blob/main/FasterRCNN_VisDrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os

# set up file paths
code_dir = '/content/drive/MyDrive/Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline'
#code_dir = '/content/drive/MyDrive/Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/data/visdrone2019-det/VisDrone/labels/val_xmls'
# move to code directory
os.chdir(code_dir)

In [ ]:
# prompt: display the files inside train_xmls folder

print(os.listdir())


['requirements.txt', 'README.md', '__init__.py', 'onnx_inference_image.py', 'inference.py', 'datasets.py', 'eval.py', 'export.py', 'inference_video.py', '.gitignore', 'LICENSE', '_config.yml', 'onnx_inference_video.py', '.git', 'data', 'data_configs', 'docs', 'example_test_data', 'models', 'notebook_examples', 'readme_images', 'torch_utils', 'utils', '.ipynb_checkpoints', '__pycache__', 'outputs', 'train.py']


In [ ]:
!unrar x '/content/drive/MyDrive/Faster_RCNN_Pretrained_Backbone_Training.rar'

Streaming output truncated to the last 5000 lines.
Extracting  Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/visdrone2019-det/VisDrone/labels/train/9999945_00000_d_0000041.txt      99%  OK 
Extracting  Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/visdrone2019-det/VisDrone/labels/train/9999945_00000_d_0000042.txt      99%  OK 
Extracting  Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/visdrone2019-det/VisDrone/labels/train/9999945_00000_d_0000043.txt      99%  OK 
Extracting  Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/visdrone2019-det/VisDrone/labels/train/9999945_00000_d_0000044.txt      99%  OK 
Extracting  Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/visdrone2019-det/VisDrone/labels/train/9999945_00000_d_0000045.txt      99%  OK 
Extracting  Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipe

In [ ]:
# Install the Requirements
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 70.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 14.5 MB/s eta 0:00:00
  Created wheel for vision_transformers: filename=vision_transforme

In [ ]:
!pip install opendatasets
!pip install pandas

import opendatasets as od
import pandas

od.download("https://www.kaggle.com/datasets/shisuiotsutsuki/visdrone2019-det")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: bijshakya077
Your Kaggle Key: ··········


100%|██████████| 1.53G/1.53G [00:53<00:00, 30.9MB/s]


In [ ]:
# %% [code]
from lxml import etree
from PIL import Image
import csv
import os

# (NOTICE #1)
# If you're OS is Mac, there's a case when '.DS_Store' file is  automatically created.
# In that case, you have to remove '.DS_Store' file through the terminal.
# Ref : http://leechoong.com/posts/2018/ds_store/
# (NOTICE #2)
# Change this path variable
IMG_PATH = "/content/drive/MyDrive/Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/data/visdrone2019-det/VisDrone/images/val"
fw = os.listdir(IMG_PATH)
# path of save xml file
save_path = '/content/drive/MyDrive/Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/data/visdrone2019-det/VisDrone/labels/val_xmls/' # keep it blank

# txt_folder is txt file root that using darknet rectbox
txt_folder = '/content/drive/MyDrive/Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/data/visdrone2019-det/VisDrone/labels/val'

# (NOTICE #3)
# Change this labels
labels = ['car', 'other']
global label
label = ''

def csvread(fn):
    with open(fn, 'r') as csvfile:
        list_arr = []
        reader = csv.reader(csvfile, delimiter=' ')

        for row in reader:
            list_arr.append(row)
    return list_arr


def convert_label(txt_file):
    global label
    for i in range(len(labels)):
        if txt_file == str(i):
            label = labels[i]
            return label

    return label

# core code = convert the yolo txt file to the x_min,x_max...


def extract_coor(txt_file, img_width, img_height):
    x_rect_mid = float(txt_file[1])
    y_rect_mid = float(txt_file[2])
    width_rect = float(txt_file[3])
    height_rect = float(txt_file[4])

    x_min_rect = ((2 * x_rect_mid * img_width) - (width_rect * img_width)) / 2
    x_max_rect = ((2 * x_rect_mid * img_width) + (width_rect * img_width)) / 2
    y_min_rect = ((2 * y_rect_mid * img_height) -
                  (height_rect * img_height)) / 2
    y_max_rect = ((2 * y_rect_mid * img_height) +
                  (height_rect * img_height)) / 2

    return x_min_rect, x_max_rect, y_min_rect, y_max_rect


for line in fw:
    root = etree.Element("annotation")

    # try debug to check your path
    img_style = IMG_PATH.split('/')[-1]
    img_name = line
    image_info = IMG_PATH + "/" + line
    img_txt_root = txt_folder + "/" + line[:-4]
    # print(img_txt_root)
    txt = ".txt"

    txt_path = img_txt_root + txt
    # print(txt_path)
    txt_file = csvread(txt_path)
    ######################################

    # read the image  information
    img_size = Image.open(image_info).size

    img_width = img_size[0]
    img_height = img_size[1]
    img_depth = Image.open(image_info).layers
    ######################################

    folder = etree.Element("folder")
    folder.text = "%s" % (img_style)

    filename = etree.Element("filename")
    filename.text = "%s" % (img_name)

    path = etree.Element("path")
    path.text = "%s" % (IMG_PATH)

    source = etree.Element("source")
    ##################source - element##################
    source_database = etree.SubElement(source, "database")
    source_database.text = "Unknown"
    ####################################################

    size = etree.Element("size")
    ####################size - element##################
    image_width = etree.SubElement(size, "width")
    image_width.text = "%d" % (img_width)

    image_height = etree.SubElement(size, "height")
    image_height.text = "%d" % (img_height)

    image_depth = etree.SubElement(size, "depth")
    image_depth.text = "%d" % (img_depth)
    ####################################################

    segmented = etree.Element("segmented")
    segmented.text = "0"

    root.append(folder)
    root.append(filename)
    root.append(path)
    root.append(source)
    root.append(size)
    root.append(segmented)

    for ii in range(len(txt_file)):
        label = convert_label(txt_file[ii][0])
        x_min_rect, x_max_rect, y_min_rect, y_max_rect = extract_coor(
            txt_file[ii], img_width, img_height)

        object = etree.Element("object")
        ####################object - element##################
        name = etree.SubElement(object, "name")
        name.text = "%s" % (label)

        pose = etree.SubElement(object, "pose")
        pose.text = "Unspecified"

        truncated = etree.SubElement(object, "truncated")
        truncated.text = "0"

        difficult = etree.SubElement(object, "difficult")
        difficult.text = "0"

        bndbox = etree.SubElement(object, "bndbox")
        #####sub_sub########
        xmin = etree.SubElement(bndbox, "xmin")
        xmin.text = "%d" % (x_min_rect)
        ymin = etree.SubElement(bndbox, "ymin")
        ymin.text = "%d" % (y_min_rect)
        xmax = etree.SubElement(bndbox, "xmax")
        xmax.text = "%d" % (x_max_rect)
        ymax = etree.SubElement(bndbox, "ymax")
        ymax.text = "%d" % (y_max_rect)
        #####sub_sub########

        root.append(object)
        ####################################################

    file_output = etree.tostring(root, pretty_print=True, encoding='UTF-8')
    # print(file_output.decode('utf-8'))
    ff = open('%s.xml' % (img_name[:-4]), 'w', encoding="utf-8")
    ff.write(file_output.decode('utf-8'))



In [ ]:
# prompt: write a python script to use device as GPU

import torch

# Check if CUDA is available
if torch.cuda.is_available():
  # Tell PyTorch to use the GPU
  device = torch.device("cuda")
  print("Using CUDA")
else:
  # Tell PyTorch to use the CPU
  device = torch.device("cpu")
  print("Using CPU")


Using CPU


In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
#!export CUDA_VISIBLE_DEVICES=0,1
#!export PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb:128"
#!python -m torch.distributed.launch --nproc_per_node=2 --use_env train.py --data data_configs/visdrone2019_det.yaml --epochs 3 --model fasterrcnn_resnet50_fpn --name visdrone_2_training --batch 16 --device 'cuda' --lr 0.1 --disable-wandb

!python train.py --data data_configs/visdrone2019_det.yaml --epochs 3 --model fasterrcnn_resnet50_fpn_v2 --name visdrone2_training --batch 4 --device 'cuda' --disable-wandb --lr 0.1 --imgsz 256

2023-12-10 21:12:03.093730: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 21:12:03.093787: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 21:12:03.093819: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 21:12:04.310924: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Not using distributed mode
device cuda
Checking Labels and images...
100% 6471/6471 [00:00<00:00, 13771.96it/s]
Checking Labels and images...
100% 548/548 [00:00<00:00, 114786.19it/s]
Number of training samples: 6471
Number of validation 

In [ ]:
!python eval.py --model fasterrcnn_resnet50_fpn_v2 --weights outputs/training/trial/best_model.pth --data data_configs/visdrone2019_det.yaml --batch 4 --device 'cuda' --disable-wandb --lr 0.1

##**FasterViT Training**

In [ ]:
!pip install fastervit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.7 MB/s eta 0:00:00


In [ ]:
from fastervit import create_model
vis_model = create_model('faster_vit_0_224',
                          pretrained=True,
                          model_path="/tmp/faster_vit_0.pth.tar")

In [ ]:
ls

_config.yml    example_test_data/  models/                  readme_images/
data/          export.py           notebook_examples/       README.md
data_configs/  inference.py        onnx_inference_image.py  requirements.txt
datasets.py    inference_video.py  onnx_inference_video.py  torch_utils/
docs/          __init__.py         outputs/                 train.py
eval.py        LICENSE             __pycache__/             utils/


In [ ]:
!export CUDA_VISIBLE_DEVICES=0,1
!torchrun --nnodes=4 --nproc_per_node=8 train.py --config configs/faster_vit_2_224_1k.yaml--model faster_vit_1_224 --tag faster_vit_1_224_exp_1 --batch-size 16 --drop-path-rate 0.2 --lr 0.05 --mesa 0.2 --model-ema --opt adamw --weight-decay 0.05 --amp --input-size 3 224 224 --data_dir "/content/drive/MyDrive/Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/data/visdrone2019-det/VisDrone"

[2023-12-11 04:48:46,488] torch.distributed.run: [WARNING] 
[2023-12-11 04:48:46,488] torch.distributed.run: [WARNING] *****************************************
[2023-12-11 04:48:46,488] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2023-12-11 04:48:46,488] torch.distributed.run: [WARNING] *****************************************
^C


In [ ]:
!python train.py \
    --model faster_vit_2_224_1k \
    --dataset '/content/drive/MyDrive/Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/data/visdrone2019-det/VisDrone' \
    --img 640 640 \
    --batch 16 \
    --epochs 3 \
    --data '/content/drive/MyDrive/Faster_RCNN_Pretrained_Backbone/src/fastercnn-pytorch-training-pipeline/data_configs/visdrone2019_det.yaml' \
    --weights '' \
    --device 'cuda'\

In [ ]:
!python train.py --model faster_vit_0_224 --imgsz 256 --batch 16 --epochs 3 --data data_configs/visdrone2019_det.yaml --device 'cuda' --disable-wandb

2023-12-11 05:13:58.582645: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 05:13:58.582705: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 05:13:58.582740: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 05:13:59.702316: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Not using distributed mode
device cuda
Checking Labels and images...
100% 6471/6471 [00:00<00:00, 14659.16it/s]
Checking Labels and images...
100% 548/548 [00:00<00:00, 128795.17it/s]
Number of training samples: 6471
Number of validation 